<a href="https://colab.research.google.com/github/karthikmandelli/frugal-labs-/blob/main/15_days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_csv("/content/SRM-DATA-FROM-JAN-1.csv")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta

In [ ]:
# Assuming you have a 'date' column and a 'temperature' column in your dataset
data['Date'] = pd.to_datetime(data['Date'])
data['AirTC'] = pd.to_numeric(data['AirTC'])


In [ ]:
data['AirPP'] = pd.to_numeric(data['AirPP'])
data['AirHP'] = pd.to_numeric(data['AirHP'])

In [ ]:
X = pd.to_datetime(data['Date']).values.reshape(-1, 1)
y = data['AirTC'].values.reshape(-1, 1)

In [ ]:
# Additional Feature Engineering (example: extracting month and day of the week)
data['month'] = data['Date'].dt.month
data['day_of_week'] = data['Date'].dt.dayofweek

X = data[['month', 'day_of_week']].values
y = data['AirTC'].values

In [ ]:
X = data[['AirPP', 'AirHP']].values
y = data['AirTC'].values

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Model Training
# Step 3: Model Training
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
##############by using means grouping 

# Step 2: Data Preprocessing
data['Date'] = pd.to_datetime(data['Date'])  # Convert datetime column to pandas datetime format
data['hour'] = data['Date'].dt.hour  # Extract hour from datetime column
data['day'] = data['Date'].dt.day  # Extract day from datetime column

# Step 3: Calculate average temperature for each hour
hourly_avg_temp = data.groupby('hour')['AirTC'].max()
#x1 = [data.groupby('hour')['AirTC'].max()-data.groupby('hour')['AirTC'].mean()]
# Step 4: Generate next 15 days' datetime at hourly intervals
next_18_days = pd.date_range(start=data['Date'].max(), periods=24*18, freq='17T')

# Step 5: Extract hour and day for the next 15 days
next_hour = [dt.hour for dt in next_18_days]
next_day = [dt.day for dt in next_18_days]

# Step 6: Create DataFrame with predictions for the next 15 days
predictions = pd.DataFrame({'datetime': next_18_days})

# Step 7: Fill in temperature predictions based on average temperature at the corresponding hour
predictions['temperature'] = [hourly_avg_temp[hour]-2.8 for hour in next_hour]

print(predictions)

               datetime  temperature
0   2023-05-19 23:56:57        28.45
1   2023-05-20 00:13:57        27.96
2   2023-05-20 00:30:57        27.96
3   2023-05-20 00:47:57        27.96
4   2023-05-20 01:04:57        27.66
..                  ...          ...
427 2023-05-25 00:55:57        27.96
428 2023-05-25 01:12:57        27.66
429 2023-05-25 01:29:57        27.66
430 2023-05-25 01:46:57        27.66
431 2023-05-25 02:03:57        27.08

[432 rows x 2 columns]


In [ ]:
SUBSET = predictions.iloc[135:165]

In [ ]:
print(SUBSET)

               datetime  temperature
135 2023-05-21 14:11:57        40.37
136 2023-05-21 14:28:57        40.37
137 2023-05-21 14:45:57        40.37
138 2023-05-21 15:02:57        40.59
139 2023-05-21 15:19:57        40.59
140 2023-05-21 15:36:57        40.59
141 2023-05-21 15:53:57        40.59
142 2023-05-21 16:10:57        40.83
143 2023-05-21 16:27:57        40.83
144 2023-05-21 16:44:57        40.83
145 2023-05-21 17:01:57        39.73
146 2023-05-21 17:18:57        39.73
147 2023-05-21 17:35:57        39.73
148 2023-05-21 17:52:57        39.73
149 2023-05-21 18:09:57        32.50
150 2023-05-21 18:26:57        32.50
151 2023-05-21 18:43:57        32.50
152 2023-05-21 19:00:57        31.64
153 2023-05-21 19:17:57        31.64
154 2023-05-21 19:34:57        31.64
155 2023-05-21 19:51:57        31.64
156 2023-05-21 20:08:57        29.75
157 2023-05-21 20:25:57        29.75
158 2023-05-21 20:42:57        29.75
159 2023-05-21 20:59:57        29.75
160 2023-05-21 21:16:57        29.55
1

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Assume the dataset contains columns 'Date' and 'Temperature' with daily temperature values for each May

# Step 2: Extract Features
data['Date'] = pd.to_datetime(data['Date'])  # Convert Date column to pandas datetime format
data['Day'] = data['Date'].dt.day  # Extract day from Date column
data['Temperature'] = data['AirTC'].astype(float)  # Convert Temperature column to float

# Step 3: Split Data
train_data = data[:-365]  # Use data from previous years as the training set
test_data = data[-365:]  # Keep the most recent year as the testing set

# Step 4: Create a SARIMA Model
model = SARIMAX(train_data['Temperature'], order=(1, 0, 0), seasonal_order=(1, 1, 1, 12))

# Step 5: Train the Model
model_fit = model.fit()

# Step 6: Evaluate the Model
predictions = model_fit.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1)

# Step 7: Make Predictions for May of the Current Year
current_year_data = pd.DataFrame({'Day': range(1, 32)})  # Assuming May has 31 days
forecast = model_fit.predict(start=len(train_data), end=len(train_data) + len(current_year_data) - 1)

# Print the forecasted temperatures for May of the current year
print(forecast)


10776    24.369287
10777    24.404844
10778    24.492291
10779    24.595803
10780    24.722269
10781    24.770942
10782    24.850048
10783    24.943133
10784    25.070573
10785    25.125999
10786    25.231383
10787    25.321383
10788    25.360277
10789    25.316916
10790    25.336917
10791    25.379328
10792    25.441510
10793    25.435307
10794    25.464824
10795    25.506735
10796    25.595450
10797    25.613003
10798    25.674792
10799    25.732606
10800    25.739227
10801    25.666004
10802    25.658635
10803    25.675858
10804    25.714623
10805    25.686963
10806    25.696737
Name: predicted_mean, dtype: float64


In [ ]:
!pip install keras


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.keras.layers import LSTM


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from matplotlib import rcParams
rcParams['figure.figsize']=10,6
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
import time
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Step 1: Data Collection
data = pd.read_csv('/content/SRM-DATA-FROM-JAN-1.csv')  # Assuming bengaluru_rainfall_data.csv contains date and rainfall columns

# Step 2: Data Preprocessing
data['Date'] = pd.to_datetime(data['Date'])  # Convert Date column to pandas datetime format
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

# Step 3: Feature Selection
features = ['Year', 'Month', 'Day']  # Adjust the features based on your dataset and domain knowledge

# Step 4: Splitting the Data
X = data[features]
y = data['RF']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Selecting a Model
model = RandomForestRegressor()

# Step 6: Training the Model
model.fit(X_train, y_train)

# Step 7: Model Evaluation
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)

print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)

# Step 8: Making Predictions


future_predictions = model.predict(future_data)

print('Future Rainfall Predictions:')
print(future_predictions)


Mean Squared Error: 364.02737978324717
Mean Absolute Error: 0.41138588171985574


NameError: ignored

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
# Step 2: Data Preprocessing
data['Date'] = pd.to_datetime(data['Date'])  # Convert date column to pandas datetime format
data['Day'] = data['Date'].dt.day  # Extract day from the date column
data['Month'] = data['Date'].dt.month  # Extract month from the date column
# Step 3: Feature Engineering
X = data[['Day', 'Month']]  # Input features (day and month)
y = data['RF']  # Target variable (rainfall)
# Step 4: Splitting the Data
X_train = X[:-15]  # Training data (all data except the last 15 days)
y_train = y[:-15]
X_test = X[-15:]  # Testing data (last 15 days)
y_test = y[-15:]
model = LinearRegression()
# Step 6: Training the Model
model.fit(X_train, y_train)
# Step 7: Model Evaluation
accuracy = model.score(X_test, y_test)
print('Model Accuracy:', accuracy)
# Step 8: Making Predictions for the Next 15 Days
next_15_days = pd.date_range(start=data['Date'].max() + pd.Timedelta(days=1), periods=24, freq='D')
next_days = next_15_days.day
next_months = next_15_days.month
new_data = pd.DataFrame({'Day': next_days, 'Month': next_months})
predictions = model.predict(new_data)
print('Next 15 days rainfall predictions:')
print(predictions)


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
 # Assuming rainfall_data.csv contains date and rainfall columns

# Step 2: Data Preprocessing
data['Date'] = pd.to_datetime(data['Date'])  # Convert Date column to pandas datetime format
data['day'] = data['Date'].dt.day  # Extract day from Date column
data['month'] = data['Date'].dt.month  # Extract month from Date column

# Step 3: Feature Engineering
X = data[['day', 'month']]  # Input features: day and month
y = data['RF']  # Target variable: rainfall

# Step 4: Splitting the Data
X_train = X[:-15]  # Use all data except the last 15 days for training
y_train = y[:-15]
X_test = X[-15:]  # Use the last 15 days for testing
y_test = y[-15:]

# Step 5: Selecting a Model
model = LinearRegression()

# Step 6: Training the Model
model.fit(X_train, y_train)

# Step 7: Model Evaluation
accuracy = model.score(X_test,predictions)
print('Model Accuracy:', -accuracy)

# Step 8: Making Predictions for Next 15 Days
next_15_days = pd.DataFrame()
next_15_days['Date'] = pd.date_range(start=data['Date'].max() + pd.DateOffset(days=1), periods=15, freq='D')  # Generate next 15 days' dates
next_15_days['day'] = next_15_days['Date'].dt.day  # Extract day from Date column
next_15_days['month'] = next_15_days['Date'].dt.month  # Extract month from Date column

predictions = model.predict(next_15_days[['day', 'month']])

# Combine dates and corresponding rainfall predictions
predictions_df = pd.DataFrame({'Date': next_15_days['Date'], 'Rainfall Prediction': predictions})

print('Rainfall Predictions for the Next 15 Days:')
print(predictions_df)


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
# Step 2: Data Preprocessing
data['Date'] = pd.to_datetime(data['Date'])  # Convert Date column to pandas datetime format
data['day'] = data['Date'].dt.day  # Extract day from Date column
data['month'] = data['Date'].dt.month  # Extract month from Date column
# Step 3: Feature Engineering
X = data[['day', 'month']]  # Input features: day and month
y = data['RF']  # Target variable: rainfall
# Step 4: Model Selection and Cross-Validation
model = LinearRegression()
scores = cross_val_score(model, X, y, cv=5)  # Perform 5-fold cross-validation
# Print cross-validation scores
print('Cross-Validation Scores:', scores)
print('Average Cross-Validation Score:', scores.mean())
# Step 5: Training the Model
model.fit(X, y)
# Step 6: Making Predictions for Next 15 Days
next_15_days = pd.DataFrame()
next_15_days['Date'] = pd.date_range(start=data['Date'].max() + pd.DateOffset(days=1), periods=15, freq='D')  # Generate next 15 days' dates
next_15_days['day'] = next_15_days['Date'].dt.day  # Extract day from Date column
next_15_days['month'] = next_15_days['Date'].dt.month  # Extract month from Date column
predictions = model.predict(next_15_days[['day', 'month']])
# Combine dates and corresponding rainfall predictions
predictions_df = pd.DataFrame({'Date': next_15_days['Date'], 'Rainfall Prediction': predictions})
print('Rainfall Predictions for the Next 15 Days:')
print(predictions_df)
accuracy = model.score(X_test,predictions)
print('Model Accuracy:', accuracy)


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
# Step 2: Data Preprocessing
data['Date'] = pd.to_datetime(data['Date'])  # Convert Date column to pandas datetime format
data['day'] = data['Date'].dt.day  # Extract day from Date column
data['month'] = data['Date'].dt.month  # Extract month from Date column
# Step 3: Feature Engineering
X = data[['day', 'month']]  # Input features: day and month
y = data['RF']  # Target variable: rainfall
# Step 4: Model Selection and Cross-Validation
model = RandomForestRegressor()
scores = cross_val_score(model, X, y, cv=5)  # Perform 5-fold cross-validation
# Print cross-validation scores
print('Cross-Validation Scores:', scores)
print('Average Cross-Validation Score:', scores.mean())
# Step 5: Training the Model
model.fit(X, y)
# Step 6: Making Predictions for Next 15 Days
next_15_days = pd.DataFrame()
next_15_days['Date'] = pd.date_range(start=data['Date'].max() + pd.DateOffset(days=1), periods=15, freq='D')  # Generate next 15 days' dates
next_15_days['day'] = next_15_days['Date'].dt.day  # Extract day from Date column
next_15_days['month'] = next_15_days['Date'].dt.month  # Extract month from Date column
predictions = model.predict(next_15_days[['day', 'month']])
# Combine dates and corresponding rainfall predictions
predictions_df = pd.DataFrame({'Date': next_15_days['Date'], 'Rainfall Prediction': predictions})
print('Rainfall Predictions for the Next 15 Days:')
print(predictions_df)
accuracy = model.score(X_test,predictions)
print('Model Accuracy:', accuracy)



In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
# Step 2: Data Preprocessing
data['Date'] = pd.to_datetime(data['Date'])  # Convert Date column to pandas datetime format
data['day'] = data['Date'].dt.day  # Extract day from Date column
data['month'] = data['Date'].dt.month  # Extract month from Date column
# Step 3: Feature Engineering
X = data[['day', 'month', 'AirPP', 'AirTC', 'AirHP']]  # Input features: day, month, pressure, temperature, humidity
y = data['RF']  # Target variable: rainfall
# Step 4: Model Selection and Cross-Validation
model = RandomForestRegressor()
scores = cross_val_score(model, X, y, cv=5)  # Perform 5-fold cross-validation
# Print cross-validation scores
print('Cross-Validation Scores:', scores)
print('Average Cross-Validation Score:', scores.mean())
# Step 5: Training the Model
model.fit(X, y)
# Step 6: Making Predictions for Next 15 Days
next_15_days = pd.DataFrame()
next_15_days['Date'] = pd.date_range(start=data['Date'].max() + pd.DateOffset(days=1), periods=15, freq='D')  # Generate next 15 days' dates
next_15_days['day'] = next_15_days['Date'].dt.day  # Extract day from Date column
next_15_days['month'] = next_15_days['Date'].dt.month  # Extract month from Date column

# Use the latest available pressure, temperature, and humidity values for predictions
latest_pressure = data['AirPP'].iloc[-1]
latest_temperature = data['AirTC'].iloc[-1]
latest_humidity = data['AirHP'].iloc[-1]

next_15_days['AirPP'] = latest_pressure
next_15_days['AirTC'] = latest_temperature
next_15_days['AirHP'] = latest_humidity
predictions = model.predict(next_15_days[['day', 'month', 'AirPP', 'AirTC', 'AirHP']])
# Combine dates and corresponding rainfall predictions
predictions_df = pd.DataFrame({'Date': next_15_days['Date'], 'Rainfall Prediction': predictions})
print('Rainfall Predictions for the Next 15 Days:')
print(predictions_df)
print('Cross-Validation Scores:', scores)
print('Average Cross-Validation Score:', scores.mean())


In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

#suming weather_data.csv contains date, pressure, temperature, humidity, and rainfall columns

# Step 2: Data Preprocessing
data['Date'] = pd.to_datetime(data['Date'])  # Convert Date column to pandas datetime format
data['day'] = data['Date'].dt.day  # Extract day from Date column
data['month'] = data['Date'].dt.month  # Extract month from Date column

# Step 3: Feature Engineering
features = ['day', 'month', 'AirPP', 'AirTC', 'AirHP']
target = 'RF'

# Scale the features and target variable
scaler = MinMaxScaler()
data[features + [target]] = scaler.fit_transform(data[features + [target]])

# Step 4: Prepare the Data for LSTM
def create_lstm_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:i + time_steps].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10  # Number of previous time steps to consider
X, y = create_lstm_dataset(data[features], data[target], time_steps)

# Split the data into training and testing sets
split_ratio = 0.8  # 80% for training, 20% for testing
split_index = int(split_ratio * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Step 5: Build and Train the LSTM Model
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(time_steps, len(features))))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=10, batch_size=32)

# Step 6: Prediction and Evaluation
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print('Train Mean Squared Error:', mse_train)
print('Test Mean Squared Error:', mse_test)

# Step 7: Making Predictions for Next 15 Days
last_sequence = data[features].tail(time_steps).values
next_15_days = pd.DataFrame(last_sequence, columns=features)

next_15_days_pred = []
for _ in range(15):
    next_pred = model.predict(np.expand_dims(next_15_days[-time_steps:].values, axis=0))
    next_15_days_pred.append(next_pred[0, 0])
    next_15_days = next_15_days.append({**dict(zip(features, next_pred[0]))}, ignore_index=True)

next_15_days[target] = scaler.inverse_transform(np.array(next_15_days_pred).reshape(-1, 1))

print('Rainfall Predictions for the Next 15 Days:')
print(next_15_days[['day', 'month', target]])


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Step 1: Data Collection
data = pd.read_csv('weather_data.csv')  # Assuming weather_data.csv contains date, pressure, temperature, humidity, and rainfall columns

# Step 2: Data Preprocessing
data['Date'] = pd.to_datetime(data['Date'])  # Convert Date column to pandas datetime format
data['day'] = data['Date'].dt.day  # Extract day from Date column
data['month'] = data['Date'].dt.month  # Extract month from Date column

# Step 3: Feature Engineering
features = ['day', 'month', 'pressure', 'temperature', 'humidity']
target = 'rainfall'

# Scale the features and target variable
scaler = MinMaxScaler()
data[features + [target]] = scaler.fit_transform(data[features + [target]])

# Step 4: Prepare the Data for LSTM
def create_lstm_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:i + time_steps].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10  # Number of previous time steps to consider
X, y = create_lstm_dataset(data[features], data[target], time_steps)

# Split the data into training and testing sets
split_ratio = 0.8  # 80% for training, 20% for testing
split_index = int(split_ratio * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Step 5: Build and Train the LSTM Model
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(time_steps, len(features))))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=10, batch_size=32)

# Step 6: Prediction and Evaluation
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print('Train Mean Squared Error:', mse_train)
print('Test Mean Squared Error:', mse_test)

# Step 7: Making Predictions for Next 15 Days
last_sequence = data[features].tail(time_steps).values
next_15_days = pd.DataFrame(last_sequence, columns=features)

next_15_days_pred = []
for _ in range(15):
    next_pred = model.predict(np.expand_dims(next_15_days[-time_steps:].values, axis=0))
    next_15_days_pred.append(next_pred[0, 0])
    next_15_days = next_15_days.append({**dict(zip(features, next_pred[0]))}, ignore_index=True)

next_15_days[target] = scaler.inverse_transform(np.array(next_15_days_pred).reshape(-1, 1))

print('Rainfall Predictions for the Next 15 Days:')
print(next_15_days[['day', 'month', target]])


#########kaggle

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("/content/rainfall in india 1901-2015.csv")

In [ ]:
data.head(75)

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna()

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
img=np.array(Image.open('../input/indian-rainfall/IndiaRainfall.png'))
fig=plt.figure(figsize=(10,10))
plt.imshow(img,interpolation='bilinear')
plt.axis('off')
plt.ioff()
plt.show()

In [33]:
import pandas as pd

  # Replace 'historical_data.csv' with your file path or data source
df = pd.read_csv("/content/SRM-DATA-FROM-JAN-1.csv")
# Convert the date column to datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Extract year, month, day, and hour from the Date column
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour

# Group the data by date and hour
grouped_data = df.groupby(['Year', 'Month', 'Day', 'Hour'])

# Calculate average temperature for each group
average_temperatures = grouped_data['AirTC'].mean()

# Iterate over the average temperatures and extract the desired time slots
predicted_temperatures = []
for i, temp in average_temperatures.iteritems():
    year, month, day, hour = i
    if hour in [1, 2, 3, 4, 5, 6]:
        time_slot = '1-6 AM'
    elif hour in [7, 8, 9, 10, 11, 12]:
        time_slot = '6-12 PM'
    elif hour in [13, 14, 15, 16, 17, 18]:
        time_slot = '12-6 PM'
    else:
        time_slot = '6-12 AM'
    predicted_temperatures.append([year, month, day, time_slot, temp])

# Create a new DataFrame with the predicted temperatures
predicted_df = pd.DataFrame(predicted_temperatures, columns=['Year', 'Month', 'Day', 'Time Slot', 'Temperature'])

# Print the predicted temperatures
print(predicted_df)


      Year  Month  Day Time Slot  Temperature
0     2022     12   13   6-12 PM    29.700000
1     2022     12   13   6-12 PM    30.240000
2     2022     12   13   6-12 PM    29.696667
3     2022     12   13   12-6 PM    29.656667
4     2022     12   13   12-6 PM    30.203333
...    ...    ...  ...       ...          ...
3439  2023      5   19   6-12 AM    30.655000
3440  2023      5   19   6-12 AM    30.290000
3441  2023      5   19   6-12 AM    30.077500
3442  2023      5   19   6-12 AM    29.846667
3443  2023      5   19   6-12 AM    29.775000

[3444 rows x 5 columns]


<ipython-input-33-a8e947c8daa4>:22: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, temp in average_temperatures.iteritems():


In [36]:
import pandas as pd
import statsmodels.api as sm

#eplace 'historical_data.csv' with your file path or data source

# Convert the date column to datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Set the date column as the index
df.set_index('Date', inplace=True)

# Create a SARIMA model
model = sm.tsa.SARIMAX(df['AirTC'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

# Fit the model to the data
model_fit = model.fit()

# Forecast the next 15 days
forecast = model_fit.forecast(steps=15)

# Print the forecasted temperatures
print(forecast)


KeyError: ignored